In [1]:
# Install Dependencies

In [2]:
!pip install -q transformers[torch] datasets

In [3]:
!pip install -q bitsandbytes trl peft


In [4]:
!pip install flash-attn --no-build-isolation

DOCLING BASED GENERATION OF DATA

In [5]:
# !pip install docling

In [6]:
# from docling.document_converter import DocumentConverter



In [7]:
# source = "https://www.JEEnet.in/wp-content/uploads/2019/07/JEE-MODEL-QUESTIONS-Part-1.pdf"
# converter = DocumentConverter()
# result = converter.convert(source)
# print(result.document.export_to_markdown())

**Hugging face dataset**

In [8]:
from datasets import load_dataset

raw_datasets = load_dataset("Likhi2003/linearalgebra_QA")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
from datasets import DatasetDict

# remove this when done debugging

dataset_dict = {"train": raw_datasets["train"].select(range(0,int(223*.8))),
                "test": raw_datasets["train"].select(range(int(223*.8),223))}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 178
    })
    test: Dataset({
        features: ['text'],
        num_rows: 45
    })
})

INST has questions and the other part has answer, we need to separate those

In [10]:
example = raw_datasets["train"][0]
print(example.keys())
print(example)

dict_keys(['text'])
{'text': '<s>[INST] Level-1: What is the matrix-vector multiplication of a 2x2 matrix A = [a11, a12] and a 1x2 vector x = [x1]? [/INST] Ax = [a11x1, a12x1] </s>'}


In [11]:
import re


seq = raw_datasets["train"]["text"]

# Regular expression to capture the INST part and the rest of the string
for sequence in seq:
    match = re.match(r"<s>\[INST\](.*?)\[/INST\](.*?)</s>", sequence)
    if match:
        inst_part = match.group(1).strip()
        rest_part = match.group(2).strip()
        print("Question:", inst_part)
        print("Answer:", rest_part)


Question: Level-1: What is the matrix-vector multiplication of a 2x2 matrix A = [a11, a12] and a 1x2 vector x = [x1]?
Answer: Ax = [a11x1, a12x1]
Question: Level-1: What is the matrix-vector multiplication of a 2x2 matrix A = [a11, a12] and a 1x2 vector x = [x1]?
Answer: Ax = [a11x1, a12x1]
Question: Level-1: If A is an nxn matrix and x is an nx1 vector, what is the value of Ax = [Ax1, Ax2, ..., Axn]?
Answer: Ax = x1v1 + x2v2 + ... + xnvn.
Question: Level-1: Prove that the matrix-vector multiplication of a square matrix A = [a11, a12, ..., a1n] and a vector x = [x1, x2, ..., xn] is commutative.
Answer: The matrix-vector multiplication of a square matrix A = [a11, a12, ..., a1n] and a vector x = [x1, x2, ..., xn] is commutative.
Question: Level-1: If A is an nxn matrix and x is an nx2 vector, what is the value of Ax = [Ax1, Ax2, ..., Axn]?
Answer: Ax = [Ax1, Ax2, ..., Axn] = [x1v1 + x2v2 + ... + xnvn, x1v2 + x2v3 + ... + xnv n, ..., x1v n + x2v n + ... + xnv n].
Question: Level-1: Prove

In [12]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [13]:
from transformers import AutoTokenizer

model_id = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Can be bf16 if your GPU supports it
    device_map="auto",  # Auto-assign devices (CPU/GPU)
    low_cpu_mem_usage=True,  # Prevents RAM overflow
    attn_implementation="flash_attention_2",  # Reduces memory usage
    load_in_4bit=True,  # Uses 4-bit quantization to reduce memory
)


# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

In [14]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 178
    })
    test: Dataset({
        features: ['text'],
        num_rows: 45
    })
})

In [15]:
from transformers import BitsAndBytesConfig
import torch

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
)
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
    attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

In [ ]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments

# path where the Trainer will save its checkpoints and logs
output_dir = 'data/zephyr-7b-sft-lora'

# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    # push_to_hub=True,
    # hub_model_id="zephyr-7b-sft-lora",
    # hub_strategy="every_save",
    # report_to="tensorboard",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
    # packing=False,
    # dataset_text_field="text",
    # max_seq_length=tokenizer.model_max_length,

)

# based on config
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

trainer = SFTTrainer(
        model=model,
        # model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=raw_datasets["train"],
        eval_dataset=raw_datasets["test"],
        tokenizer=tokenizer,
        peft_config=peft_config,
    )

In [ ]:
train_result = trainer.train()

In [ ]:
metrics = train_result.metrics
max_train_samples = training_args.max_train_samples if training_args.max_train_samples is not None else len(raw_datasets["train"])
metrics["train_samples"] = min(max_train_samples, len(raw_datasets["train"]))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir, load_in_4bit=True, device_map="auto")

In [ ]:
import torch

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {"role": "user", "content": "Generate 10 linear algebra questions"}
]

# prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])